In [ ]:
import string
import re
import os
import nltk
import pandas as pd
import numpy as np
import json
import glob
import gzip
import tensorflow as tf
from tensorflow import keras
import time

import matplotlib.pyplot as plt

SEED = 1013
np.random.seed(SEED)

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import pickle
from sklearn.naive_bayes import MultinomialNB
import pandas as pd 
import numpy as np 
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from scipy import sparse
import os
import pickle
import emoji
from wordcloud import WordCloud

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.util import ngrams

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize, sent_tokenize
stop_words = set(stopwords.words('english'))

In [ ]:
import collections

df = pd.read_csv('path_human_annotated_africanamerican_tweets')
dff = pd.read_csv('path_human_annotated_africanamerican_tweets2')
dfff = pd.read_csv('path_human_annotated_non-africanamerican_tweets')

In [ ]:
races = df['race'].to_list() + dff['race'].to_list() + dfff['race'].to_list()

In [ ]:
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+' #remove mentions
pat2 = r'https?://[^ ]+' #remove hyperlinks
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner_updated(text):
    soup = BeautifulSoup(text, 'lxml') # remove html tags
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?") #remove byte order marks
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    stripped = re.sub(r'\@w+','',stripped)
    lower_case = stripped.lower()
    lower_case = emoji.demojize(lower_case, language='en')
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled) #remove hashtag
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

In [ ]:
tweets = df['tweet'].to_list() + dff['tweet'].to_list() + dfff['tweet'].to_list()
cleaned_tweets = []
for tweet in tweets:
    cleaned_tweets.append(tweet_cleaner_updated(tweet))

In [ ]:
pos_code_map={'CC':'A','CD':'B','DT':'C','EX':'D','FW':'E','IN':'F','JJ':'G','JJR':'H','JJS':'I','LS':'J','MD':'K','NN':'L','NNS':'M',
'NNP':'N','NNPS':'O','PDT':'P','POS':'Q','PRP':'R','PRP$':'S','RB':'T','RBR':'U','RBS':'V','RP':'W','SYM':'X','TO':'Y','UH':'Z',
'VB':'1','VBD':'2','VBG':'3','VBN':'4','VBP':'5','VBZ':'6','WDT':'7','WP':'8','WP$':'9','WRB':'@'}

In [ ]:
code_pos_map = {v: k for k, v in  pos_code_map.items()}

In [ ]:
#abbrivation converters
def convert(tag):
    try:
        code=pos_code_map[tag]
    except:
        code='?'
    return code
def inv_convert(code):
    try:
        tag=code_pos_map[code]
    except:
        tag='?'
    return tag

In [ ]:
#POS tag converting
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag, word_tokenize
def pos_tags(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    return "".join(convert(tag) for (word, tag) in nltk.pos_tag(text_processed))
def text_pos_inv_convert(text):
    return "-".join(inv_convert(c.upper()) for c in text)

In [ ]:
train_corpus,test_corpus,train_labels,test_labels = train_test_split(cleaned_tweets,races,stratify=races,test_size=0.25,random_state=1)

In [ ]:
pos_for_tweets = []
for tweet in train_corpus:
    text_pos = lambda x: pos_tags(tweet)
    pos_for_tweets.append(text_pos(tweet))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#v = CountVectorizer(analyzer='char',ngram_range=(1, 3))

v = TfidfVectorizer(stop_words = 'english', analyzer='char',ngram_range=(1, 1))
ngrams = v.fit_transform(pos_for_tweets)


In [ ]:
t = TfidfVectorizer(stop_words = 'english',max_features=10000,ngram_range=(1, 1))

In [ ]:
tfidf_train = t.fit_transform(train_corpus)

In [ ]:
# do the below on test data

In [ ]:
pos_for_tweets_test = []
for tweet in x_test:
    text_pos = lambda x: pos_tags(tweet)
    pos_for_tweets_test.append(text_pos(tweet))

In [ ]:
ngrams_test = v.transform(pos_for_tweets_test)

In [ ]:
tfidf_test = t.transform(x_test)

In [ ]:
from scipy.sparse import hstack
test_test = hstack((tfidf_test,ngrams_test))

In [ ]:
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(x_test, y_test)
print(result)